In [ ]:
import csaf.config as cconf
import csaf.system as csys

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_multiagent_config.toml")

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
def air_collision_condition(ctraces):
        """ground collision premature termnation condition
        TODO: I had to alter the terminating conditions to support this expression
        """
        # get the aircraft states
        sa, sb = ctraces['planta']['states'], ctraces['plantb']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 10

In [ ]:
# TODO: add to docker image
!pip install onnxruntime

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 3.0]

In [ ]:
# set the scenario states
my_system.set_state('planta', [400.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_system.set_state('plantb', [400.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 2000.0, 0.0, 6000.0, 9.0])

In [ ]:
# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, 
                                 terminating_conditions=air_collision_condition,
                                 show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
# plot the lateral trajectories
def add_airplane(posa, ax, label):
    t = mpl.markers.MarkerStyle(marker='^')
    a = np.arctan2(*(posa[-1]-posa[2])) + np.pi/2
    t._transform = t.get_transform().rotate_deg(np.rad2deg(a))
    plt.scatter(*posa[-1], marker=t, s=400, label=label)


posa = np.array(trajs['planta'].states)[:, 9:11]
posb = np.array(trajs['plantb'].states)[:, 9:11]

fig, ax = plt.subplots(figsize=(10, 10))

plt.scatter(*posa.T, linewidth=2, s=10, c=np.linspace(0, 1, len(posa)))
plt.scatter(*posb.T, linewidth=2, s=10, c=np.linspace(0, 1, len(posa)))

add_airplane(posa, plt.gca, label='F-16 A')
add_airplane(posb, plt.gca, label='F-16 B')

plt.title('ACAS Maneuver Eastward / Northward Displacement')
plt.xlabel('Northward Displacement (m)')
plt.ylabel('Eastward Displacement (m)')
plt.legend()
plt.grid()

In [ ]:
# plot the distances
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(trajs['planta'].times, np.linalg.norm(posa - posb, axis=1))
plt.title("ACAS Maneuver Distance Between Aircraft")
plt.yscale('log')
plt.xlabel('Time (s)')
plt.ylabel('Distance (m)')
plt.grid()